In [1]:
# Install required packages if not already installed
!pip install requests elasticsearch

# Use requests to get Elasticsearch cluster information
import requests
import json

# Make a request to the Elasticsearch cluster
response = requests.get("http://localhost:9200")

# Print the response in a formatted way
cluster_info = response.json()
print(json.dumps(cluster_info, indent=2))

# Extract the build_hash value
build_hash = cluster_info["version"]["build_hash"]
print(f"version.build_hash: {build_hash}")


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
{
  "name": "2e0448a9a91c",
  "cluster_name": "docker-cluster",
  "cluster_uuid": "eZme00LUTwGiH_9C_XEd5A",
  "version": {
    "number": "8.4.3",
    "build_flavor": "default",
    "build_type": "docker",
    "build_hash": "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date": "2022-10-04T07:17:24.662462378Z",
    "build_snapshot": false,
    "lucene_version": "9.3.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}
version.build_hash: 42f05b9372a9a4a470db3b52817899b99a76ee73


In [2]:
# Get the data
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Print the first document to see its structure
print(json.dumps(documents[0], indent=2))

# Connect to Elasticsearch
from elasticsearch import Elasticsearch

# Connect to your local Elasticsearch instance
es = Elasticsearch("http://localhost:9200")

# Check if the connection is successful
print(f"Connected to Elasticsearch: {es.ping()}")

# Create index with mapping
index_name = "faq_index"

# Define the mapping - 'course' as keyword, others as text
mapping = {
    "mappings": {
        "properties": {
            "course": {"type": "keyword"},
            "question": {"type": "text"},
            "text": {"type": "text"},
            "section": {"type": "text"}
        }
    }
}

# Delete the index if it already exists (optional)
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Deleted existing index: {index_name}")

# Create the index with the mapping
es.indices.create(index=index_name, body=mapping)
print(f"Created index: {index_name}")

# Index the documents
for i, doc in enumerate(documents):
    es.index(index=index_name, id=i, document=doc)
    if i % 100 == 0:  # Print progress every 100 documents
        print(f"Indexed {i} documents")

print(f"Indexed all {len(documents)} documents")

{
  "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
  "section": "General course-related questions",
  "question": "Course - When will the course start?",
  "course": "data-engineering-zoomcamp"
}
Connected to Elasticsearch: True
Deleted existing index: faq_index
Created index: faq_index
Indexed 0 documents
Indexed 100 documents
Indexed 200 documents
Indexed 300 documents
Indexed 400 documents
Indexed 500 documents
Indexed 600 documents
Indexed 700 documents
Indexed 800 documents
Indexed 900 documents
Indexed all 948 documents


In [3]:
# Define the search query
search_query = {
    "query": {
        "multi_match": {
            "query": "How do execute a command on a Kubernetes pod?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

# Execute the search
results = es.search(index=index_name, body=search_query)

# Print the top result with its score
top_result = results["hits"]["hits"][0]
top_score = top_result["_score"]
print(f"Top result score: {top_score}")
print(f"Top result question: {top_result['_source']['question']}")
print(f"Top result text: {top_result['_source']['text']}")

Top result score: 44.50556
Top result question: How do I debug a docker container?
Top result text: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


In [5]:
# Define the filtered query
filter_query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question", "text"]
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    },
    "size": 3
}

# Execute the filtered search
filtered_results = es.search(index=index_name, body=filter_query)

# Print all three results
for i, hit in enumerate(filtered_results["hits"]["hits"]):
    print(f"Result {i+1}:")
    print(f"  Question: {hit['_source']['question']}")
    print(f"  Score: {hit['_score']}")
    print()

# Specifically print the third result
third_result = filtered_results["hits"]["hits"][2]
print(f"Third result question: {third_result['_source']['question']}")

Result 1:
  Question: How do I copy files from my local machine to docker container?
  Score: 22.931826

Result 2:
  Question: How do I copy files from a different folder into docker container’s working directory?
  Score: 19.161318

Result 3:
  Question: How do I debug a docker container?
  Score: 18.34669

Third result question: How do I debug a docker container?


In [6]:
# Build context entries from the filtered results
context_entries = []
for hit in filtered_results["hits"]["hits"]:
    doc = hit["_source"]
    context_entry = f"Q: {doc['question']}\nA: {doc['text']}"
    context_entries.append(context_entry)

# Join the context entries with double linebreaks
context = "\n\n".join(context_entries)

# Define the question
question = "How do copy a file to a Docker container?"

# Build the prompt using the template
prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:

{context}
""".strip()

# Calculate the length of the prompt
prompt_length = len(prompt)
print(f"Prompt length: {prompt_length}")

# Print the prompt to see its content
print("\nPrompt content:")
print(prompt)

Prompt length: 1447

Prompt content:
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
In the Dockerfile, you can provide the folder containing the files that you want to cop

In [7]:
# Install tiktoken if not already installed
!pip install tiktoken

# Calculate tokens
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
token_count = len(tokens)
print(f"Token count: {token_count}")

# Optional: Decode a few tokens to see what they represent
print("\nSample token decoding:")
for i in range(min(5, len(tokens))):
    print(f"Token {i}: {tokens[i]} -> {encoding.decode_single_token_bytes(tokens[i])}")


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Token count: 321

Sample token decoding:
Token 0: 63842 -> b"You're"
Token 1: 261 -> b' a'
Token 2: 4165 -> b' course'
Token 3: 14029 -> b' teaching'
Token 4: 29186 -> b' assistant'


In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
token_count = len(tokens)
print(f"Token count: {token_count}")